In [74]:
import os
import sys
home_dir = "../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd

In [75]:
patho_with_dbnsfp_df = pd.read_csv(home_dir+f"models/dbnsfp/postprocessed_outputs/patho_and_likelypatho.tsv", sep="\t")
print(patho_with_dbnsfp_df.columns)
print(patho_with_dbnsfp_df.shape)
print(patho_with_dbnsfp_df["class"].value_counts())

Index(['clinvar_id', 'gene_symbol', 'gene_id', 'snp_id', 'chrom_acc_version',
       'chrom_pos', 'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos',
       'wt', 'mut', 'class', 'chrom', 'SIFT_score', 'Polyphen2_HVAR_score',
       'MetaRNN_score', 'REVEL_score', 'MVP_score', 'CADD_raw_score',
       'integrated_fitCons_score', 'phyloP17way_primate_score',
       'phastCons17way_primate_score', 'bStatistic_score'],
      dtype='object')
(6467, 24)
Likely-pathogenic    4162
Pathogenic           2305
Name: class, dtype: int64


In [76]:
popu_freq_with_dbnsfp_df = pd.read_csv(home_dir+f"models/aa_common/datasets_population_freq/popu_freq_with_dbnsfp.tsv", sep="\t")
print(popu_freq_with_dbnsfp_df.columns)
print(popu_freq_with_dbnsfp_df.shape)
print(popu_freq_with_dbnsfp_df["class"].value_counts())

Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'chrom', 'SIFT_score',
       'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score',
       'CADD_raw_score', 'integrated_fitCons_score',
       'phyloP17way_primate_score', 'phastCons17way_primate_score',
       'bStatistic_score', 'n_methods_having_preds'],
      dtype='object')
(741768, 26)
Singleton     419107
Ultra-rare    283849
Rare           25005
Common         13807
Name: class, dtype: int64


In [77]:
popu_freq_df = popu_freq_with_dbnsfp_df.copy()
# popu_freq_df = popu_freq_df[(popu_freq_df["class"]=="Common") | (popu_freq_df["class"]=="Rare")]
# popu_freq_df = popu_freq_df[(popu_freq_df["class"]!="Singleton")]

unique_prots = patho_with_dbnsfp_df["prot_acc_version"].unique()
print("#-of uniqie proteins: ", unique_prots.shape[0])

data = []
for i, prot in enumerate(unique_prots):
    
    z = patho_with_dbnsfp_df[patho_with_dbnsfp_df["prot_acc_version"]==prot].copy()
    x = popu_freq_df[popu_freq_df["prot_acc_version"]==prot].copy()

    
    if x.shape[0]==0: continue

    remaining = 0
    y = x[(x["class"]=="Common") | (x["class"]=="Rare")]
    if y.shape[0] >= z.shape[0]:
        sampled = y.sample(z.shape[0], random_state=29)
    else:
        sampled = y
    remaining = z.shape[0] - sampled.shape[0]

    if remaining!=0:
        y = x[(x["class"]=="Ultra-rare")]
        if y.shape[0] >= remaining:
            remaining_sampled = y.sample(remaining, random_state=29)
        else:
            remaining_sampled = y
        
        remaining = remaining - remaining_sampled.shape[0]
        sampled = pd.concat([sampled, remaining_sampled])
              
    if remaining!=0:
        y = x[x["class"]=="Singleton"].copy()
        if y.shape[0] >= remaining:
            remaining_sampled = y.sample(remaining, random_state=29)
        else:
            remaining_sampled = y

        sampled = pd.concat([sampled, remaining_sampled])

    print(f"{i}/{unique_prots.shape[0]}", prot, z.shape[0], x.shape[0], sampled.shape[0])
    data.append(sampled)

neutral_df = pd.concat(data)

print(neutral_df.shape)
print(neutral_df.columns)
print(neutral_df["class"].value_counts())

#-of uniqie proteins:  1430
7/1430 NP_009193.2 2 23 2
8/1430 NP_004276.2 1 186 1
9/1430 NP_005948.3 8 72 8
10/1430 NP_055689.1 13 86 13
11/1430 NP_009203.2 3 36 3
12/1430 NP_000076.2 2 114 2
13/1430 NP_002991.2 16 25 16
14/1430 NP_000469.3 72 59 59
15/1430 NP_001782.1 2 5 2
16/1430 NP_005817.1 1 30 1
17/1430 NP_000138.2 1 61 1
18/1430 NP_065184.2 3 77 3
19/1430 NP_060116.2 1 56 1
20/1430 NP_116182.2 1 41 1
21/1430 NP_000301.1 7 38 7
22/1430 NP_683763.2 2 34 2
23/1430 NP_006507.2 22 39 22
24/1430 NP_060620.2 2 69 2
25/1430 NP_000365.3 3 43 3
26/1430 NP_116145.1 11 34 11
27/1430 NP_060209.4 8 9 8
28/1430 NP_000089.1 5 92 5
29/1430 NP_000783.2 1 26 1
30/1430 NP_777596.2 2 116 2
31/1430 NP_002624.2 2 71 2
32/1430 NP_000320.1 11 61 11
33/1430 NP_000007.1 33 42 33
34/1430 NP_891993.1 1 15 1
35/1430 NP_000960.2 3 15 3
36/1430 NP_036375.1 2 25 2
37/1430 NP_001909.4 10 42 10
38/1430 NP_057042.2 2 26 2
39/1430 NP_006504.2 1 39 1
40/1430 NP_001010898.1 2 71 2
41/1430 NP_004965.1 8 30 8
42/1430 NP

In [83]:
info_cols = ['clinvar_id', 'snp_id', 
             'chrom_acc_version', 'chrom', 'chrom_pos', 'ref_allele', 'alt_allele', 
             'prot_acc_version', 'prot_pos', 'wt', 'mut', 'class']
method_cols = ['SIFT_score', 'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score', 'CADD_raw_score', 
               'integrated_fitCons_score', 'phyloP17way_primate_score', 'phastCons17way_primate_score', 'bStatistic_score']
columns = info_cols + method_cols

neutral_df["clinvar_id"] = None
neutral_df = neutral_df[columns]
print(neutral_df.shape)
print(neutral_df.columns)
print(neutral_df["class"].value_counts())
neutral_df["prot_acc_version"].unique().shape[0]

(6050, 22)
Index(['clinvar_id', 'snp_id', 'chrom_acc_version', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos', 'wt', 'mut',
       'class', 'SIFT_score', 'Polyphen2_HVAR_score', 'MetaRNN_score',
       'REVEL_score', 'MVP_score', 'CADD_raw_score',
       'integrated_fitCons_score', 'phyloP17way_primate_score',
       'phastCons17way_primate_score', 'bStatistic_score'],
      dtype='object')
Ultra-rare    3582
Rare          1176
Singleton      767
Common         525
Name: class, dtype: int64


1395

In [79]:
patho_df = patho_with_dbnsfp_df[columns]
result_df = pd.concat([patho_df, neutral_df])
print(result_df.shape)
print(result_df.columns)
print(result_df["class"].value_counts())

(12517, 22)
Index(['clinvar_id', 'snp_id', 'chrom_acc_version', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos', 'wt', 'mut',
       'class', 'SIFT_score', 'Polyphen2_HVAR_score', 'MetaRNN_score',
       'REVEL_score', 'MVP_score', 'CADD_raw_score',
       'integrated_fitCons_score', 'phyloP17way_primate_score',
       'phastCons17way_primate_score', 'bStatistic_score'],
      dtype='object')
Likely-pathogenic    4162
Ultra-rare           3582
Pathogenic           2305
Rare                 1176
Singleton             767
Common                525
Name: class, dtype: int64


In [80]:
out_filepath = home_dir+f"models/aa_common/datasets_pathogenicity/patho_likelypatho_neutral_dbnsfp"
result_df.to_csv(out_filepath+".tsv", sep="\t", index=False)

# Creating merged fasta document ...
protein_acc_list = list(result_df["prot_acc_version"].unique())
print(len(protein_acc_list))
from utils.ncbi_proteins import create_combined_fasta
create_combined_fasta(protein_acc_list, out_filepath+".fasta", home_dir)

1430
0 NP_001381642.1 Already existis
1 NP_542172.2 Already existis
2 NP_001164006.1 Already existis
3 NP_002065.1 Already existis
4 NP_000806.2 Already existis
5 NP_003027.1 Already existis
6 NP_002608.1 Already existis
7 NP_009193.2 Already existis
8 NP_004276.2 Already existis
9 NP_005948.3 Already existis
10 NP_055689.1 Already existis
11 NP_009203.2 Already existis
12 NP_000076.2 Already existis
13 NP_002991.2 Already existis
14 NP_000469.3 Already existis
15 NP_001782.1 Already existis
16 NP_005817.1 Already existis
17 NP_000138.2 Already existis
18 NP_065184.2 Already existis
19 NP_060116.2 Already existis
20 NP_116182.2 Already existis
21 NP_000301.1 Already existis
22 NP_683763.2 Already existis
23 NP_006507.2 Already existis
24 NP_060620.2 Already existis
25 NP_000365.3 Already existis
26 NP_116145.1 Already existis
27 NP_060209.4 Already existis
28 NP_000089.1 Already existis
29 NP_000783.2 Already existis
30 NP_777596.2 Already existis
31 NP_002624.2 Already existis
32 NP_0